## Análisis de CSV

In [32]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import unicodedata

In [5]:
df = pd.read_csv('final/all_data.csv', delimiter=';', encoding='utf-8-sig')

In [6]:
df.head()

,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
0,00-01-0158-46,01-312,CIUDAD CAPITAL,ZONA 1,COLEGIO TECNICO PROGRESISTA CETECPRO,2A. AVENIDA 3-59,22512759,AMADO SALOMON FLORES PEREZ,IRMA AMPARO TOLEDO HERNANDEZ,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,DOBLE,DIARIO(REGULAR),GUATEMALA NORTE
1,00-01-0173-46,01-301,CIUDAD CAPITAL,ZONA 1,INSTITUTO NORMAL PARA SEÑORITAS CENTRO AMERICA,1A CALLE 2-64,22323424,LUCRECIA MARISOL CERMEÑO GONZÁLEZ,INGRID MARIELA ESPINA ORELLANA,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),GUATEMALA NORTE
2,00-01-0174-46,01-307,CIUDAD CAPITAL,ZONA 1,ESCUELA NACIONAL CENTRAL DE FORMACION SECRETARIAL,12 AVENIDA 9-27,22322985,NORMA ARACELY PALOMO FRANCO DE DIAZ,FRANCISCO RENÉ CONTRERAS AQUINO,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),GUATEMALA NORTE
3,00-01-0176-46,01-301,CIUDAD CAPITAL,ZONA 1,INSTITUTO NACIONAL DE BACHILLERATO EN COMPUTACION,3A CALLE 15-45,22202223,LUCRECIA MARISOL CERMEÑO GONZÁLEZ,FREDY HUMBERTO GONZÁLEZ SANTISTEBAN,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),GUATEMALA NORTE
4,00-01-0178-46,01-403,CIUDAD CAPITAL,ZONA 1,ESCUELA NACIONAL CENTRAL DE CIENCIAS COMERCIALES,10A. AVENIDA 9-42,22320914,CARLOS HUMBERTO GONZÁLEZ DE LEÓN,ESWIN NOE ROSALES LÓPEZ,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,NOCTURNA,DIARIO(REGULAR),GUATEMALA NORTE


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6599 entries, 0 to 6598
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CODIGO           6599 non-null   object
 1   DISTRITO         6599 non-null   object
 2   DEPARTAMENTO     6599 non-null   object
 3   MUNICIPIO        6599 non-null   object
 4   ESTABLECIMIENTO  6599 non-null   object
 5   DIRECCION        6597 non-null   object
 6   TELEFONO         6553 non-null   object
 7   SUPERVISOR       6599 non-null   object
 8   DIRECTOR         6573 non-null   object
 9   NIVEL            6599 non-null   object
 10  SECTOR           6599 non-null   object
 11  AREA             6599 non-null   object
 12  STATUS           6599 non-null   object
 13  MODALIDAD        6599 non-null   object
 14  JORNADA          6599 non-null   object
 15  PLAN             6599 non-null   object
 16  DEPARTAMENTAL    6599 non-null   object
dtypes: object(17)
memory usage: 876.6

In [7]:
print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")

Filas: 6599, Columnas: 17


In [8]:
missing = df.isnull().sum().to_frame(name='missing_count')
missing['missing_percent'] = (missing['missing_count'] / len(df)) * 100
missing.sort_values(by='missing_percent', ascending=False)

,missing_count,missing_percent
TELEFONO,46,0.697075
DIRECTOR,26,0.393999
DIRECCION,2,0.030308
CODIGO,0,0.000000
DISTRITO,0,0.000000
ESTABLECIMIENTO,0,0.000000
MUNICIPIO,0,0.000000
DEPARTAMENTO,0,0.000000
SUPERVISOR,0,0.000000
NIVEL,0,0.000000


### Revisión de columna Teléfono 

In [15]:
print("Ejemplos de TELEFONO:")
print(df['TELEFONO'].dropna().unique()[:10])

# Ver cuántos valores tienen caracteres no numéricos
telefonos_problematicos = df['TELEFONO'] = df['TELEFONO'].astype(str) 
df['TELEFONO_non_numeric'] = df['TELEFONO'].str.contains(r'\D', regex=True)

print("\nTotal con caracteres no numéricos (guiones, espacios, puntos, etc.):")
print(df['TELEFONO_non_numeric'].sum())

# Ver ejemplos con problemas
print("\nEjemplos con caracteres no numéricos:")
print(df[df['TELEFONO_non_numeric']]['TELEFONO'].unique())

Ejemplos de TELEFONO:
['22512759' '22323424' '22322985' '22202223' '22320914' '58068368'
 '42201837' '22384684' '22329406' '22325522']

Total con caracteres no numéricos (guiones, espacios, puntos, etc.):
83

Ejemplos con caracteres no numéricos:
['nan' '24342036-24123888' '24314989-24310955' '66355887-66312603'
 '52660000-78323083' '79649696-78739432' '78739432-79649696'
 '78392709-78396701-78396702' '78880429-78880419' '56769964-77663283'
 '56769964-7766328' '77663283-77663283' '77375333-42470723' '4.21039E+15'
 '4.2811E+15' '79540830-79540909' '79504027-79504028' '31193946-46843572'
 '79484391-41051885' '45261770-46007362' '79480085-50307050'
 '79524680-79524680' '53886336-53886336' '79497500-78209540'
 '31089418-53230377' '42601313-79476691' '53660792-55969273'
 '79477890-79477891' '79224958-79223033']


In [17]:
# Ver qué caracteres no numéricos aparecen en esos teléfonos

problematicos = df[df['TELEFONO_non_numeric']]['TELEFONO'].dropna().astype(str)

# Extraer los caracteres no numéricos de todos los teléfonos
no_digit_chars = ''.join(re.findall(r'\D', ''.join(problematicos)))

# Contar cuáles son más comunes
Counter(no_digit_chars)

Counter({'n': 92, 'a': 46, '-': 35, '.': 3, 'E': 3, '+': 3})

In [18]:
telefonos_solo_digitos = df['TELEFONO'].astype(str).str.replace(r'\D+', '', regex=True)

longitudes = telefonos_solo_digitos.str.len()

conteo_por_longitud = longitudes.value_counts().sort_index()

print("Distribución por cantidad de dígitos en TELEFONO:")
print(conteo_por_longitud)

Distribución por cantidad de dígitos en TELEFONO:
TELEFONO
0       46
1        1
2        1
4        1
6        3
7       18
8     6495
15       1
16      32
24       1
Name: count, dtype: int64


In [ ]:
ejemplos_dobles = df[telefonos_solo_digitos.str.len() > 8]['TELEFONO'].unique()

for tel in ejemplos_dobles:
    print(tel)

24342036-24123888
24314989-24310955
66355887-66312603
52660000-78323083
79649696-78739432
78739432-79649696
78392709-78396701-78396702
78880429-78880419
56769964-77663283
56769964-7766328
77663283-77663283
77375333-42470723
79540830-79540909
79504027-79504028
31193946-46843572
79484391-41051885
45261770-46007362
79480085-50307050
79524680-79524680
53886336-53886336
79497500-78209540
31089418-53230377
42601313-79476691
53660792-55969273
79477890-79477891
79224958-79223033


In [13]:
ejemplos_dobles = df[telefonos_solo_digitos.str.len() < 8]['TELEFONO'].unique()

for tel in ejemplos_dobles:
    print(tel)

2232068
2223228
2232379
5899624
nan
2437011
5510735
5364736
40
4215928
3033
783928
4225675
4140069
5899378
4.2811E+15
574479
0
4085613
533290
4167076
3127247
5388633
7844007


### Revisión de columna Director

In [21]:
# Ver cantidad de valores únicos y nulos
print("Cantidad total de filas:", len(df))
print("Valores únicos en 'DIRECTOR':", df['DIRECTOR'].nunique())
print("Valores nulos en 'DIRECTOR':", df['DIRECTOR'].isnull().sum())

Cantidad total de filas: 6599
Valores únicos en 'DIRECTOR': 3863
Valores nulos en 'DIRECTOR': 26


In [22]:
# Ver valores más repetidos (top 10)
print("\nTop 10 nombres más frecuentes:")
print(df['DIRECTOR'].value_counts(dropna=False).head(10))


Top 10 nombres más frecuentes:
DIRECTOR
NaN                                 26
MARÍA DOLORES PÉREZ TUCHÁN          10
HÉCTOR REYNALDO GÓMEZ AGUILAR        9
SILVIA MARLENI ALVARADO GUARDADO     8
MELVIN RAFAEL REYES LÓPEZ            8
RONI ERNESTO RECINOS ESTRADA         8
SONIA JOSEFINA MORALES CAXAJ         8
GERSON ARELY PUZ SAL                 7
MARCO TULIO VARGAS ALVARADO          7
JHONATAN EDUARDO SAC CAYAX           7
Name: count, dtype: int64


In [23]:
# Ver ejemplos únicos (sin repetir)
print("\nAlgunos ejemplos únicos:")
print(df['DIRECTOR'].dropna().unique()[:10])


Algunos ejemplos únicos:
['IRMA AMPARO TOLEDO HERNANDEZ' 'INGRID MARIELA ESPINA ORELLANA'
 'FRANCISCO RENÉ CONTRERAS AQUINO' 'FREDY HUMBERTO GONZÁLEZ SANTISTEBAN'
 'ESWIN NOE ROSALES LÓPEZ' 'JULIO CESAR SANTOS CAMPOS'
 'JULIO ERNESTO GONZÁLEZ HERNÁNDEZ' 'SILVIA LETICIA PERÉN POYÓN'
 'ELUBIA ANAYTE SIGUENZA MONTERROSO' 'NANCY ALEIDA BANCES ARÉVALO']


In [24]:
# Ver longitudes de los nombres
print("\nDistribución de longitud de los nombres:")
print(df['DIRECTOR'].dropna().str.len().value_counts().sort_index())


Distribución de longitud de los nombres:
DIRECTOR
1       1
2       2
3       6
4       3
5       2
7       1
8       5
10      1
12      1
13      9
14     11
15     22
16     30
17     47
18     86
19     83
20    122
21    134
22    189
23    267
24    356
25    489
26    600
27    765
28    751
29    660
30    586
31    465
32    309
33    226
34    146
35     65
36     46
37     19
38     24
39     18
40      8
41      8
42      5
43      2
44      1
49      1
51      1
Name: count, dtype: int64


In [37]:
# Asegurar que todo sea string
df['DIRECTOR'] = df['DIRECTOR'].astype(str)

# Ver nombres muy cortos (< 10 caracteres)
print("📏 Nombres muy cortos (<10 caracteres):")
nombres_cortos = df[df['DIRECTOR'].str.len() < 6]['DIRECTOR'].unique()
print(f"Total: {len(nombres_cortos)}")
print(nombres_cortos)  

# Ver nombres muy largos (> 40 caracteres)
print("\n📏 Nombres muy largos (>40 caracteres):")
nombres_largos = df[df['DIRECTOR'].str.len() > 40]['DIRECTOR'].unique()
print(f"Total: {len(nombres_largos)}")
print(nombres_largos) 

📏 Nombres muy cortos (<10 caracteres):
Total: 6
['---' 'nan' '----' '-----' 'X' '--']

📏 Nombres muy largos (>40 caracteres):
Total: 17
['EVA ANGELINA RAMIREZ IXCAMPARIJ DE RAMÍREZ'
 'SONIA LETICIA MACHUCA RODRÍGUEZ DE PARDUCCI'
 'MARGOTH LETICIA ESTURBAN HERRARTE DE LOPEZ'
 'BETTY CONSUELO GARCIA BERNAL DE CASTAÑEDA'
 'INGRID JOHANA PANIAGUA MARIN DE FERNANDEZ'
 'ANDREA VANESSA ALDANA VILLAGRAN DE CASTRO'
 'DORA AMANDA MARTÍNEZ ESTRADA DE SAQUILMER'
 'MARINA DE LOS ANGELES HERNÁNDEZ GÓMEZ DE BARRIENTOS'
 'KRISZTYN HELKIA SABRINA MORALES BERREONDO'
 'BELMA JEANNETH SECAIDA CONTRERAS DE MAYÉN'
 'LILIAN ARGENTINA PANIAGUA AJÍN DE CASTILLO'
 'JESSICA JEANNETTE SAGASTUME HICKS DE TZORIN.'
 'CLAUDIA MARLENI ELÍAS COJULUN DE CIFUENTES'
 'THELMA ASUCENA MENDOZA MENDOZA DE CARRILLO'
 'THELMA KARINA QUIÑÓNEZ RODAS DE RODRÍGUEZ'
 'MAJIB JHESSIEL YAZDIE WADY JOZ LANDAVERRY VASQUEZ'
 'MARÍA VICTORIA GERÓNIMO ESCALANTE DE MÉNDEZ']


In [ ]:
# Revisar inconsistencias: mayúsculas/minúsculas
print("\nInconsistencias de capitalización:")
df['DIRECTOR_lower'] = df['DIRECTOR'].str.lower().str.strip()
capitalization_conflicts = df.groupby('DIRECTOR_lower')['DIRECTOR'].nunique()
multi_format_names = capitalization_conflicts[capitalization_conflicts > 1]
print(f"Total nombres con distintas capitalizaciones: {len(multi_format_names)}")
print(multi_format_names.head(10))


Inconsistencias de capitalización:
Total nombres con distintas capitalizaciones: 0
Series([], Name: DIRECTOR, dtype: int64)


In [35]:
# Revisar tildes vs sin tildes
def remove_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFD', text)
                   if unicodedata.category(c) != 'Mn')

df['DIRECTOR_normalized'] = df['DIRECTOR'].str.upper().str.strip().apply(remove_accents)

accent_conflicts = df.groupby('DIRECTOR_normalized')['DIRECTOR'].nunique()
accent_variants = accent_conflicts[accent_conflicts > 1]

print("\nNombres que aparecen con y sin tildes (o variantes acentuadas):")
print(f"Total con variantes acentuadas: {len(accent_variants)}")
print(accent_variants)


Nombres que aparecen con y sin tildes (o variantes acentuadas):
Total con variantes acentuadas: 37
DIRECTOR_normalized
ANAHI DEL PILAR HERNANDEZ CHOCHOM         3
ANGEL FELIPE LARIOS MELENDEZ              2
ANGELICA DEL ROSARIO PATZAN QUISQUE       2
BLANCA ROSA HERNANDEZ LOPEZ               2
CARLA ELIZABETH MONTERROSO GOMEZ          2
CESAR FERNANDO ESCOBAR ZEPEDA             2
CLAUDIA PATRICIA DAVILA RAMIREZ           2
DALIDA ANAYANZA HERNANDEZ RAMIREZ         2
DARWIN ENRIQUE ALVAREZ MORAN              2
DIEGO DANIEL DIAZ                         2
ELIAS ISRAEL LOPEZ MIRANDA                2
EMMA ETELVINA ORELLANA RAMIREZ            2
FRANCISCO REVOLORIO LOPEZ                 2
GERALD PETER SCHAEFFER GARCIA             2
HECTOR ENRIQUE ZUNIGA ORELLANA            2
HERLINDO ARTIGA MARROQUIN                 2
ISABEL CRISTINA BALCAZAR VASQUEZ          2
JOSE ANTONIO MARTINEZ ORDONEZ             2
JOSE DOMINGO PEREZ MESIAS                 2
JOSUE ESTRADA Y ESTRADA                   2


### Análisis variable dirección

In [38]:
# Aseguramos que sea texto
df['DIRECCION'] = df['DIRECCION'].astype(str)

# Nulos y únicos
print("Estado general:")
print(f"Total de filas: {len(df)}")
print(f"Nulos: {df['DIRECCION'].isnull().sum()}")
print(f"Valores únicos: {df['DIRECCION'].nunique()}")

Estado general:
Total de filas: 6599
Nulos: 0
Valores únicos: 4434


In [39]:
# Longitudes de las direcciones
print("\nDistribución de longitud de caracteres:")
print(df['DIRECCION'].str.len().value_counts().sort_index().head(15))


Distribución de longitud de caracteres:
DIRECCION
1       2
3       2
5       1
6      15
7       7
8       8
9      13
10     20
11     37
12     84
13    200
14    219
15    206
16    284
17    242
Name: count, dtype: int64


In [49]:
# Ver ejemplos muy cortos (< 10 caracteres)
print("\nDirecciones muy cortas:")
print(df[df['DIRECCION'].str.len() < 4]['DIRECCION'].unique())


Direcciones muy cortas:
['nan' '.']


In [41]:
# Buscar patrones comunes
print("\nPalabras comunes:")
palabras = df['DIRECCION'].str.upper().str.findall(r'\b\w+\b')
from collections import Counter
conteo_palabras = Counter()
for lista in palabras:
    conteo_palabras.update(lista)
for palabra, freq in conteo_palabras.most_common(10):
    print(f"{palabra}: {freq}")



Palabras comunes:
ZONA: 3049
CALLE: 2068
1: 1706
AVENIDA: 1628
BARRIO: 929
2: 928
ALDEA: 880
COLONIA: 797
3: 772
EL: 760


In [42]:
# Buscar símbolos raros
print("\n Caracteres no alfabéticos comunes:")
no_letras = ''.join(df['DIRECCION'].dropna().astype(str))
caracteres = Counter(re.findall(r'[^a-zA-Z0-9\s]', no_letras))
print(caracteres.most_common(10))


 Caracteres no alfabéticos comunes:
[('.', 3530), ('-', 3474), (',', 2189), ('"', 455), ('Ó', 364), ('Í', 174), ('Á', 154), ('É', 143), ('Ñ', 33), ('ª', 23)]


### Análisis variable Establecimiento

In [50]:
# Asegurar que todo sea string
df['ESTABLECIMIENTO'] = df['ESTABLECIMIENTO'].astype(str)

# 1. Valores únicos y nulos
print("Total de registros:", len(df))
print("Nulos:", df['ESTABLECIMIENTO'].isnull().sum())
print("Valores únicos:", df['ESTABLECIMIENTO'].nunique())

Total de registros: 6599
Nulos: 0
Valores únicos: 3786


In [51]:
# 2. Top 10 establecimientos más repetidos
print("\nTop 10 nombres más frecuentes:")
print(df['ESTABLECIMIENTO'].value_counts().head(10))


Top 10 nombres más frecuentes:
ESTABLECIMIENTO
INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA                        286
INSTITUTO NACIONAL DE EDUCACIÓN DIVERSIFICADA                        102
CENTRO DE EDUCACIÓN EXTRAESCOLAR -CEEX-                               32
INSTITUTO DE EDUCACION DIVERSIFICADA POR COOPERATIVA DE ENSEÑANZA     22
INSTITUTO DE EDUCACIÓN DIVERSIFICADA POR COOPERATIVA DE ENSEÑANZA     20
INSTITUTO DIVERSIFICADO POR COOPERATIVA                               20
ESCUELA NORMAL DE EDUCACION FISICA                                    18
INSTITUTO DIVERSIFICADO POR COOPERATIVA DE ENSEÑANZA                  14
LICEO CANADIENSE SOCIEDAD ANONIMA                                     13
ESCUELA NACIONAL DE CIENCIAS COMERCIALES                              13
Name: count, dtype: int64


In [52]:
# 3. Longitudes del texto
print("\nDistribución de longitud de los nombres:")
print(df['ESTABLECIMIENTO'].str.len().value_counts().sort_index().head(15))


Distribución de longitud de los nombres:
ESTABLECIMIENTO
3      1
4      7
5      1
6      1
11     4
12    16
13    31
14    60
15    43
16    65
17    37
18    70
19    58
20    68
21    83
Name: count, dtype: int64


In [53]:
# 4. Ver algunos ejemplos únicos
print("\nEjemplos de nombres:")
print(df['ESTABLECIMIENTO'].unique()[:10])


Ejemplos de nombres:
['COLEGIO TECNICO PROGRESISTA CETECPRO'
 'INSTITUTO NORMAL PARA SEÑORITAS CENTRO AMERICA'
 'ESCUELA NACIONAL CENTRAL DE FORMACION SECRETARIAL'
 'INSTITUTO NACIONAL DE BACHILLERATO EN COMPUTACION'
 'ESCUELA NACIONAL CENTRAL DE CIENCIAS COMERCIALES'
 'ESCUELA NORMAL PARA MAESTROS DE EDUCACION MUSICAL JESUS MARIA ALVARADO'
 'ESCUELA NACIONAL DE ADMINISTRACION PUBLICA LIC. MANUEL GREGORIO LOPEZ SANTIAGO'
 "INSTITUTO NORMAL 'CASA CENTRAL'"
 'LICEO TECNOLOGICO GUATEMALA DE LA ASUNCIÓN'
 'INSTITUTO MIXTO COACTEMALAN']


In [55]:
def normalizar_establecimiento(nombre):
    nombre = nombre.upper().strip()
    nombre = ''.join(c for c in unicodedata.normalize('NFD', nombre) if unicodedata.category(c) != 'Mn')  # sin tildes
    nombre = nombre.replace('"', '').replace('.', '').replace('  ', ' ')
    return nombre

df['ESTABLECIMIENTO_NORMALIZADO'] = df['ESTABLECIMIENTO'].apply(normalizar_establecimiento)

# Ver casos con más de una variante
grupo = df.groupby('ESTABLECIMIENTO_NORMALIZADO')['ESTABLECIMIENTO'].nunique()
variantes = grupo[grupo > 1]

print("\nEstablecimientos con variantes por formato (mayúsculas/tildes):")
print(variantes)


Establecimientos con variantes por formato (mayúsculas/tildes):
ESTABLECIMIENTO_NORMALIZADO
ACADEMIA COMERCIAL E INSTITUTO LOURDES                 2
CENTRO DE APRENDIZAJE BILINGUE NO 1                    2
CENTRO DE DESARROLLO EDUCATIVO SAN JOSE                2
CENTRO DE EDUCACION CREATIVA TOSCANA                   2
CENTRO DE EDUCACION EXTRAESCOLAR -CEEX-                2
                                                      ..
LICEO TECNOLOGICO MAYA DE GUATEMALA                    2
PROGRAMA NACIONAL DE EDUCACION ALTERNATIVA -PRONEA-    2
PROGRAMA NACIONAL DE EDUCACION ALTERNATIVA, PRONEA     3
TECNOLOGICO MODERNO CENTRO EDUCACIONAL                 2
TECNOLOGICO PREUNIVERSITARIO                           2
Name: ESTABLECIMIENTO, Length: 510, dtype: int64


In [58]:
# Nombres con menos de 6 caracteres
cortos = df[df['ESTABLECIMIENTO'].str.len() < 7]['ESTABLECIMIENTO'].unique()
print("Nombres con menos de 6 caracteres:")
print(f"Total: {len(cortos)}")
print(cortos[:10])  # muestra los primeros 10

Nombres con menos de 6 caracteres:
Total: 4
['#NAME?' 'INED' 'ICA' 'IMEBI']


In [62]:
print("\nPosibles inconsistencias semánticas (abreviaciones):")

# Lista de abreviaciones a buscar (las verdaderas, como palabra completa)
abreviaciones = ['COL.', 'INST.', 'ESCU.', 'BÁS.', 'PRIV.', 'NAC.', 'MIX.', 'PROF.', 'INED']

# Usamos expresiones regulares con delimitadores de palabra \b
conteo_abreviaciones = Counter()
ejemplos_abreviaciones = {}

for abrev in abreviaciones:
    patron = rf'\b{re.escape(abrev)}\b'
    encontrados = df[df['ESTABLECIMIENTO'].str.upper().str.contains(patron, na=False, regex=True)]
    conteo_abreviaciones[abrev] = len(encontrados)
    ejemplos_abreviaciones[abrev] = encontrados['ESTABLECIMIENTO'].unique()[:5]

# Mostrar resultados
print("Abreviaciones exactas encontradas:")
for abrev, count in conteo_abreviaciones.items():
    print(f"{abrev}: {count} apariciones")


Posibles inconsistencias semánticas (abreviaciones):
Abreviaciones exactas encontradas:
COL.: 0 apariciones
INST.: 0 apariciones
ESCU.: 0 apariciones
BÁS.: 0 apariciones
PRIV.: 0 apariciones
NAC.: 0 apariciones
MIX.: 0 apariciones
PROF.: 0 apariciones
INED: 41 apariciones


In [65]:
# Mostrar ejemplos por abreviación
for abrev, ejemplos in ejemplos_abreviaciones.items():
    print(f"\nEjemplos con '{abrev}':")
    for ejemplo in ejemplos:
        print(f"- {ejemplo}")


Ejemplos con 'COL.':

Ejemplos con 'INST.':

Ejemplos con 'ESCU.':

Ejemplos con 'BÁS.':

Ejemplos con 'PRIV.':

Ejemplos con 'NAC.':

Ejemplos con 'MIX.':

Ejemplos con 'PROF.':

Ejemplos con 'INED':
- INED M. A. HERMENEGILDO DE JESÚS PEREIRA DEL CID
- INED METROPOLITANO DEL SUR
- INED ULEW KOTZ'I'J (TIERRA DE LAS FLORES)
- INED INSTITUTO NACIONAL DE EDUCACIÓN DIVERSIFICADA
- INSTITUTO NACIONAL DE EDUCACIÓN DIVERSIFICADA -INED-


In [ ]:
def normalizar_nombre(nombre):
    if pd.isna(nombre): return ''
    nombre = nombre.upper().strip()
    nombre = unicodedata.normalize('NFD', nombre)
    nombre = ''.join(c for c in nombre if unicodedata.category(c) != 'Mn')  
    nombre = re.sub(r'["\(\)\-]', '', nombre)  
    nombre = re.sub(r'\s+', ' ', nombre)       
    nombre = nombre.replace('.', '')           
    return nombre.strip()

# Aplicar normalización
df['ESTABLECIMIENTO_NORMALIZADO'] = df['ESTABLECIMIENTO'].apply(normalizar_nombre)

In [ ]:
agrupados = df.groupby('ESTABLECIMIENTO_NORMALIZADO')['ESTABLECIMIENTO'].nunique()
duplicados_ocultos = agrupados[agrupados > 1]

print(f"Nombres distintos que se vuelven iguales tras normalizar: {len(duplicados_ocultos)}")

# Ver algunos ejemplos
for nombre_norm in duplicados_ocultos.head(10).index:
    variantes = df[df['ESTABLECIMIENTO_NORMALIZADO'] == nombre_norm]['ESTABLECIMIENTO'].unique()
    print(f"\nVariantes del establecimiento '{nombre_norm}':")
    for v in variantes:
        print(f"- {v}")

Nombres distintos que se vuelven iguales tras normalizar: 530

Variantes del establecimiento 'ACADEMIA COMERCIAL E INSTITUTO LOURDES':
- ACADEMIA COMERCIAL E INSTITUTO LOURDES
- ACADEMIA COMERCIAL E INSTITUTO "LOURDES"

Variantes del establecimiento 'CENTRO DE APRENDIZAJE BILINGUE NO 1':
- CENTRO DE APRENDIZAJE BILINGÜE NO. 1
- CENTRO DE APRENDIZAJE BILINGUE NO. 1

Variantes del establecimiento 'CENTRO DE DESARROLLO EDUCATIVO SAN JOSE':
- CENTRO DE DESARROLLO EDUCATIVO SAN "JOSÉ"
- CENTRO DE DESARROLLO EDUCATIVO SAN JOSE

Variantes del establecimiento 'CENTRO DE EDUCACION CREATIVA TOSCANA':
- CENTRO DE EDUCACION CREATIVA TOSCANA
- CENTRO DE EDUCACIÓN CREATIVA TOSCANA

Variantes del establecimiento 'CENTRO DE EDUCACION EXTRAESCOLAR CEEX':
- CENTRO DE EDUCACION EXTRAESCOLAR -CEEX-
- CENTRO DE EDUCACIÓN EXTRAESCOLAR - CEEX
- CENTRO DE EDUCACIÓN EXTRAESCOLAR CEEX
- CENTRO DE EDUCACIÓN EXTRAESCOLAR -CEEX-

Variantes del establecimiento 'CENTRO DE EDUCACION EXTRAESCOLAR CEEX DEPARTAMENTAL':


### Análisis variable supervisor 

In [69]:
# Asegurar que sea string
df['SUPERVISOR'] = df['SUPERVISOR'].astype(str)

# Estado general
print("Estado general:")
print(f"Total de registros: {len(df)}")
print(f"Nulos (NaN o vacíos): {(df['SUPERVISOR'] == 'nan').sum()}")
print(f"Valores únicos: {df['SUPERVISOR'].nunique()}")

Estado general:
Total de registros: 6599
Nulos (NaN o vacíos): 0
Valores únicos: 599


In [70]:
# Top supervisores más frecuentes
print("\nTop 10 nombres más frecuentes:")
print(df['SUPERVISOR'].value_counts().head(10))


Top 10 nombres más frecuentes:
SUPERVISOR
MIGUEL ANGEL ARMAS ROCHA                          190
CARLOS HUMBERTO GONZÁLEZ DE LEÓN                  160
JUAN ENRIQUE MARTINEZ SOLANO                      106
REMY ARTURO SINAY GUDIEL                           94
ELSA YADIRA MARTINEZ PEREZ                         84
IDALIA DEL ROSARIO LOPEZ SANDOVAL DE PAIZ          81
MILTON ALONSO ALVAREZ FUENTES                      81
ELENA ELIZABETH SUCHITE GARNICA DE QUINTANILLA     78
LUDVIN RICARDO URRUTIA LORENTI                     77
JUAN FRANCISCO GODOY DAVILA                        74
Name: count, dtype: int64


In [72]:
# Longitud de los nombres
longitudes = df['SUPERVISOR'].str.len()
print("\n📏 Distribución de longitud")
print(longitudes.value_counts().sort_index())


📏 Distribución de longitud
SUPERVISOR
14     32
15     17
16     18
17     57
18    126
19    131
20    143
21    115
22    109
23    182
24    532
25    387
26    621
27    492
28    536
29    740
30    546
31    326
32    408
33    145
34     69
35    149
36     73
37     56
38     58
39    111
40     16
41    123
42     87
43     15
44     24
45      8
46     88
47     29
50     30
Name: count, dtype: int64


In [75]:
# 5. Ejemplos únicos
print("\nEjemplos únicos:")
print(df['SUPERVISOR'].unique()[:10])


Ejemplos únicos:
['AMADO SALOMON FLORES PEREZ' 'LUCRECIA MARISOL CERMEÑO GONZÁLEZ'
 'NORMA ARACELY PALOMO FRANCO DE DIAZ' 'CARLOS HUMBERTO GONZÁLEZ DE LEÓN'
 'AURA MARINA GARRIDO MORALES' 'LESLIE AZUCENA MONZON TECUN'
 'MARIANO DOMINGO ESTRADA TELETOR' 'ZUHILEM YESENIA ESTRADA ORTIZ'
 'SILVIA EUGENIA VALDEZ MONTERROSO DE CARBALLO'
 'IXMUCANE ALDANA VILLAVICENCIO']


In [78]:
def normalizar_nombre(nombre):
    nombre = nombre.upper().strip()
    nombre = unicodedata.normalize('NFD', nombre)
    nombre = ''.join(c for c in nombre if unicodedata.category(c) != 'Mn')  # sin tildes
    nombre = re.sub(r'[\"\'\-\.]', '', nombre)
    nombre = re.sub(r'\s+', ' ', nombre)
    return nombre.strip()

df['SUPERVISOR_NORMALIZADO'] = df['SUPERVISOR'].apply(normalizar_nombre)

agrupados = df.groupby('SUPERVISOR_NORMALIZADO')['SUPERVISOR'].nunique()
duplicados = agrupados[agrupados > 1]

print(f"\nSupervisores con variantes por formato: {len(duplicados)}")

for nombre_norm in duplicados.head(10).index:
    variantes = df[df['SUPERVISOR_NORMALIZADO'] == nombre_norm]['SUPERVISOR'].unique()
    print(f"\nVariantes de '{nombre_norm}':")
    for v in variantes:
        print(f"- {v}")


Supervisores con variantes por formato: 8

Variantes de 'AUGUSTO MANUEL ANDRES SEBASTIAN':
- AUGUSTO MANUEL ANDRÉS SEBASTIÁN
- AUGUSTO MANUEL ANDRES SEBASTIAN

Variantes de 'CARLOS HUMBERTO GONZALEZ DE LEON':
- CARLOS HUMBERTO GONZÁLEZ DE LEÓN
- CARLOS HUMBERTO GONZALEZ DE LEON

Variantes de 'FIDEL OMAR HERNANDEZ RAMIREZ':
- FIDEL OMAR HERNÁNDEZ RAMÍREZ
- FIDEL OMAR HERNANDEZ RAMIREZ

Variantes de 'JOSE DE JESUS CAMPOSECO MENDOZA':
- JOSÉ DE JESÚS CAMPOSECO MÉNDOZA
- JOSÉ DE JESÚS CAMPOSECO MENDOZA

Variantes de 'JUAN CARLOS MORALES GONZALEZ':
- JUAN CARLOS MORALES GONZALEZ
- JUAN CARLOS MORALES GONZÁLEZ

Variantes de 'JUAN JOSE TOBAR TEBALAN':
- JUAN JOSE TOBAR TEBALAN
- JUAN JOSÉ TOBAR TEBALAN

Variantes de 'JULIO RENE NOTZ CUTZAL':
- JULIO RENÉ NOTZ CUTZAL
- JULIO RENE NOTZ CUTZAL

Variantes de 'MIGUEL AJPOP VASQUEZ':
- MIGUEL AJPOP VÁSQUEZ
- MIGUEL AJPOP VASQUEZ


### Análisis de variables categóricas

In [79]:
# Lista de columnas categóricas que mencionaste
cat_cols = ['NIVEL', 'SECTOR', 'AREA', 'STATUS', 'MODALIDAD', 'JORNADA', 'PLAN', 'DEPARTAMENTAL']

for col in cat_cols:
    print(f"\nAnálisis de columna: {col}")
    print("-" * 40)
    print(f"Valores únicos (sin normalizar): {df[col].nunique()}")
    print("Valores más comunes:")
    print(df[col].value_counts(dropna=False).head(10))
    
    # Valores únicos ordenados alfabéticamente
    print("\nValores únicos (ordenados):")
    print(sorted(df[col].dropna().unique()))
    
    # Nulos
    print(f"\nNulos: {df[col].isnull().sum()} (vacíos o NaN)")


Análisis de columna: NIVEL
----------------------------------------
Valores únicos (sin normalizar): 1
Valores más comunes:
NIVEL
DIVERSIFICADO    6599
Name: count, dtype: int64

Valores únicos (ordenados):
['DIVERSIFICADO']

Nulos: 0 (vacíos o NaN)

Análisis de columna: SECTOR
----------------------------------------
Valores únicos (sin normalizar): 4
Valores más comunes:
SECTOR
PRIVADO        5417
OFICIAL         874
COOPERATIVA     213
MUNICIPAL        95
Name: count, dtype: int64

Valores únicos (ordenados):
['COOPERATIVA', 'MUNICIPAL', 'OFICIAL', 'PRIVADO']

Nulos: 0 (vacíos o NaN)

Análisis de columna: AREA
----------------------------------------
Valores únicos (sin normalizar): 3
Valores más comunes:
AREA
URBANA             5249
RURAL              1349
SIN ESPECIFICAR       1
Name: count, dtype: int64

Valores únicos (ordenados):
['RURAL', 'SIN ESPECIFICAR', 'URBANA']

Nulos: 0 (vacíos o NaN)

Análisis de columna: STATUS
----------------------------------------
Valores únicos 